```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
\newcommand{\tightlist}{}
```

```{=latex}
\title{Jupyter for DevOps}
\author{Moshe Zadka -- https://cobordism.com}
\date{}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

```{=latex}
\begin{frame}
\frametitle{Acknowledgement of Country}

Belmont (in San Francisco Bay Area Peninsula)

Ancestral homeland of the Ramaytush Ohlone people

\end{frame}
```

## SSH

### SSH with Paramiko

In [39]:
import paramiko
connect_params = dict(username="user", port=5022, look_for_keys=False, key_filename="build/client_rsa_key")

### Run a command

```{=latex}
\begin{frame}
\frametitle{Run Command with Paramiko}[fragile]
```

In [45]:
client = paramiko.SSHClient()
client.set_missing_host_key_policy(
    paramiko.client.WarningPolicy)
client.connect("localhost", **connect_params)
res = client.exec_command("ls")
files = res[1].read().decode("ascii").splitlines()
files

['some_file', 'another_file']

```{=latex}
\end{frame}
```

### Loop

## Cloud

### Configuring `boto3`

### Uploading an S3 file

### Looping

## GitLab

### Configuring the client

### Modifying one project

### Looping

## Summary

* Iterate
* Document
* Share

```{=latex}
\end{document}
```